Dataframes Needed:
- Live games (Teams playing, scores, results)
- Box scores (Player stats)
- Lead tracker

Home Page
- Live game scores, Youtube videos of highlights

Game Page
- Box scores
- Matchup stats (lead tracker, team %s)
- Youtube highlight

Visualizations
- [Blank]

In [1]:
import nbapy
import datetime
import warnings
import pandas as pd
import dateutil.parser
from nbapy import game
from nbapy.scoreboard import Scoreboard
pd.set_option('max_columns', None)

In [2]:
test_gameid = '0021901318'
test_date = '08-13-2020'

### Get Games Function

In [8]:
def get_games(datestring):
    """
    Input:
        datestring (str) - date of NBA games to grab
    
    Output:
        games (pandas df) - dataframe of NBA games, results, and teams playing
        
    """
    date = dateutil.parser.parse(datestring)
    scoreboard = Scoreboard(date.month, date.day, date.year)
    
    # Grab today's game information
    game_header = scoreboard.game_header()
    line_score = scoreboard.line_score()
    
    # Simplify columns
    games = game_header[['GAME_SEQUENCE', 'GAME_ID', 'GAME_STATUS_TEXT', 
                         'LIVE_PERIOD', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID']]
    score = line_score[['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'PTS']]
    
    # Dictionary of game values
    team_info = score.set_index('TEAM_ID')['TEAM_ABBREVIATION'].to_dict()
    team_pts = score.set_index('TEAM_ID')['PTS'].to_dict()
    
    # Map to games info
    games['HOME_TEAM_ABBREVIATION'] = games.HOME_TEAM_ID.map(team_info)
    games['VISITOR_TEAM_ABBREVIATION'] = games.VISITOR_TEAM_ID.map(team_info)
    games['HOME_TEAM_PTS'] = games.HOME_TEAM_ID.map(team_pts)
    games['VISITOR_TEAM_PTS'] = games.VISITOR_TEAM_ID.map(team_pts)
    
    # Get rid of warnings
    warnings.filterwarnings("ignore")
    
    return games

games = get_games(test_date)
games

,GAME_SEQUENCE,GAME_ID,GAME_STATUS_TEXT,LIVE_PERIOD,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_ABBREVIATION,VISITOR_TEAM_ABBREVIATION,HOME_TEAM_PTS,VISITOR_TEAM_PTS
0,1,0021901308,Final,4,1610612738,1610612764,BOS,WAS,90,96
1,2,0021901310,Final,4,1610612747,1610612758,LAL,SAC,122,136
2,3,0021901311,Final,4,1610612763,1610612749,MEM,MIL,119,106
3,4,0021901313,Final,4,1610612756,1610612742,PHX,DAL,128,102
4,5,0021901314,Final,4,1610612762,1610612759,UTA,SAS,118,112
5,6,0021901309,Final,4,1610612751,1610612757,BKN,POR,133,134
6,7,0021901312,Final,4,1610612753,1610612740,ORL,NOP,133,127


### Get Live Games

In [14]:
def get_livegames():
    """
    Input:
        None
        
    Output:
        live_games (pandas df) - Dataframe showing results for today's NBA games
    """
    today = datetime.datetime.now()
    today_string = today.strftime("%m-%d-%Y")
    live_games = get_games(today_string)
    return live_games

live_games = get_livegames()
live_games

,GAME_SEQUENCE,GAME_ID,GAME_STATUS_TEXT,LIVE_PERIOD,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_ABBREVIATION,VISITOR_TEAM_ABBREVIATION,HOME_TEAM_PTS,VISITOR_TEAM_PTS
0,1,0021901318,Final,4,1610612761,1610612743,TOR,DEN,117.0,109.0
1,2,0021901316,Final,4,1610612754,1610612748,IND,MIA,109.0,92.0
2,3,0021901317,6:30 pm ET,0,1610612746,1610612760,LAC,OKC,NaN,NaN
3,4,0021901315,9:00 pm ET,0,1610612745,1610612755,HOU,PHI,NaN,NaN


### Get Box Scores

In [5]:
def get_boxscore(game_id):
    """
    Input:
        game_id (str) - Game ID to pull box scores
    
    Output:
        box_score (pandas df) - Dataframe containing the box score results of Game ID
        
    """
    box_score = game.BoxScore(game_id).players_stats()
    return box_score

boxscore = get_boxscore(test_gameid)
boxscore.head()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,0021901318,1610612743,DEN,Denver,203924,Jerami Grant,F,,18:18,1.0,6.0,0.167,0.0,3.0,0.000,0.0,0.0,0.0,1.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0
1,0021901318,1610612743,DEN,Denver,200794,Paul Millsap,F,,15:30,4.0,8.0,0.500,2.0,3.0,0.667,0.0,0.0,0.0,1.0,2.0,3.0,2.0,2.0,1.0,1.0,2.0,10.0,-4.0
2,0021901318,1610612743,DEN,Denver,203999,Nikola Jokic,C,,10:12,1.0,1.0,1.000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,1.0,1.0,2.0,0.0,0.0,1.0,0.0,2.0,-6.0
3,0021901318,1610612743,DEN,Denver,1628470,Torrey Craig,G,,14:58,1.0,6.0,0.167,1.0,3.0,0.333,0.0,0.0,0.0,1.0,3.0,4.0,3.0,0.0,0.0,1.0,0.0,3.0,-6.0
4,0021901318,1610612743,DEN,Denver,1627750,Jamal Murray,G,,10:12,4.0,5.0,0.800,3.0,3.0,1.000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,11.0,-6.0


### Get Team Stats

In [6]:
def get_teamstats(game_id):
    """
    Input:
        game_id (str) - Game ID to pull team stats
        
    Output:
        team_stats (pandas df) - Dataframe showing the teams percentages
    
    """
    team_stats = game.BoxScore(game_id).team_stats()
    return team_stats

team_stats = get_teamstats(test_gameid)
team_stats

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,0021901318,1610612761,Raptors,TOR,Toronto,240:00,45,90,0.500,18,35,0.514,9,14,0.643,13,38,51,27,8,5,19,24,117,8.0
1,0021901318,1610612743,Nuggets,DEN,Denver,240:00,36,87,0.414,16,38,0.421,21,23,0.913,9,32,41,27,9,11,15,15,109,-8.0


### Get Lead Tracker

In [7]:
def get_linescore(game_id):
    """
    Input:
        game_id (str) - Game ID to pull line score
    
    Ouput:
        line_score (pandas df) - Dataframe showing the points by quarter
    
    """
    line_score = game.Info(game_id).line_score()
    return line_score

line_score = get_linescore(test_gameid)
line_score

,GAME_DATE_EST,GAME_SEQUENCE,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY_NAME,TEAM_NICKNAME,TEAM_WINS_LOSSES,PTS_QTR1,PTS_QTR2,PTS_QTR3,PTS_QTR4,PTS_OT1,PTS_OT2,PTS_OT3,PTS_OT4,PTS_OT5,PTS_OT6,PTS_OT7,PTS_OT8,PTS_OT9,PTS_OT10,PTS
0,2020-08-14T00:00:00,1,0021901318,1610612743,DEN,Denver,Nuggets,46-27,23,27,26,33,0,0,0,0,0,0,0,0,0,0,109
1,2020-08-14T00:00:00,1,0021901318,1610612761,TOR,Toronto,Raptors,53-19,31,27,27,32,0,0,0,0,0,0,0,0,0,0,117
